In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/minor project/train.csv")
test = pd.read_csv("/content/drive/MyDrive/minor project/test.csv")

In [ ]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [ ]:
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
tsvd = TruncatedSVD(n_components = 50)

question_title = tfidf.fit_transform(train["question_title"].values)
question_title_test = tfidf.transform(test["question_title"].values)
question_title = tsvd.fit_transform(question_title)
question_title_test = tsvd.transform(question_title_test)

question_body = tfidf.fit_transform(train["question_body"].values)
question_body_test = tfidf.transform(test["question_body"].values)
question_body = tsvd.fit_transform(question_body)
question_body_test = tsvd.transform(question_body_test)

answer = tfidf.fit_transform(train["answer"].values)
answer_test = tfidf.transform(test["answer"].values)
answer = tsvd.fit_transform(answer)
answer_test = tsvd.transform(answer_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
train["len_user_name"]= train.question_user_name.apply(lambda x : 1 if len(x)<5 else 0)
test["len_user_name"]= test.question_user_name.apply(lambda x : 1 if len(x)<5 else 0)

In [ ]:
train["cat_host"]= train["category"]+train["host"]+str(train["len_user_name"])
test["cat_host"]= test["category"]+test["host"]+str(test["len_user_name"])


category_means_map = train.groupby("len_user_name")[target_cols].mean().T.to_dict()
category_te = train["len_user_name"].map(category_means_map).apply(pd.Series)
category_te_test = test["len_user_name"].map(category_means_map).apply(pd.Series)

In [ ]:
!!!!!!!!!!!!!!!!!have to look above cell

In [ ]:
train_features = np.concatenate([question_title, question_body, answer, category_te.values
                                ], axis = 1)
test_features = np.concatenate([question_title_test, question_body_test, answer_test, category_te_test.values
                               ], axis = 1)

In [ ]:
pd.DataFrame(train_features)

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0.050953,0.019622,0.015049,-0.030194,0.017487,-0.015410,-0.020561,-0.006403,-0.006526,-0.007622,...,0.800433,0.922117,0.652417,0.955280,0.967822,0.847912,0.482664,0.129433,0.502955,0.909968
1,0.129142,0.134065,0.000254,-0.016784,-0.003482,-0.013626,0.075270,0.134270,0.039296,-0.034783,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
2,0.016268,-0.000742,-0.002396,-0.018425,0.008679,0.023436,-0.000924,-0.009807,-0.003908,-0.003361,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
3,0.044964,-0.026072,-0.011511,-0.032375,0.000074,-0.001507,0.020164,-0.054167,0.039004,-0.036494,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
4,0.113850,-0.136120,0.148736,-0.019870,-0.247015,0.047714,0.092136,-0.013954,-0.034836,0.081699,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,0.022873,-0.005829,-0.005486,-0.026554,0.017472,0.044890,0.009789,-0.022599,-0.003157,0.004410,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
6075,0.035490,-0.020488,-0.014868,0.000941,0.026480,0.031786,0.000274,-0.013531,-0.010503,0.010166,...,0.800433,0.922117,0.652417,0.955280,0.967822,0.847912,0.482664,0.129433,0.502955,0.909968
6076,0.021710,-0.014063,-0.011348,-0.004147,0.013230,-0.002554,0.016449,-0.017314,0.004799,-0.001303,...,0.799849,0.925940,0.655212,0.960826,0.968756,0.855774,0.479043,0.130836,0.502389,0.907977
6077,0.026464,-0.013366,0.000934,-0.014119,0.019129,-0.001978,0.013817,-0.037159,-0.026941,-0.037060,...,0.800433,0.922117,0.652417,0.955280,0.967822,0.847912,0.482664,0.129433,0.502955,0.909968


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import KFold
#from keras.callbacks.callbacks import EarlyStopping
from scipy.stats import spearmanr

In [ ]:
num_folds = 10
fold_scores = []
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 42)
test_preds = np.zeros((len(test_features), len(target_cols)))
for train_index, val_index in kf.split(train_features):
    train_X = train_features[train_index, :]
    train_y = train[target_cols].iloc[train_index]
    
    val_X = train_features[val_index, :]
    val_y = train[target_cols].iloc[val_index]

In [ ]:
    model = Sequential([
        Dense(1024, input_shape=(train_features.shape[1],)),
        Activation('relu'),
        Dense(512),
        Activation('relu'),
        Dense(len(target_cols)),
        Activation('sigmoid'),
    ])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy')

In [ ]:
model.fit(train_X, train_y, epochs = 50, validation_data=(val_X, val_y))#, callbacks = [es])
preds = model.predict(val_X)

overall_score = 0
for col_index, col in enumerate(target_cols):
        overall_score += spearmanr(preds[:, col_index], val_y[col].values).correlation/len(target_cols)
        print(col, spearmanr(preds[:, col_index], val_y[col].values).correlation)
        fold_scores.append(overall_score)

test_preds += model.predict(test_features)/num_folds
    
print(fold_scores)

Epoch 1/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3335 - val_loss: 0.3952
Epoch 2/50
171/171 [==============================] - 2s 12ms/step - loss: 0.3322 - val_loss: 0.3976
Epoch 3/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3306 - val_loss: 0.4008
Epoch 4/50
171/171 [==============================] - 2s 12ms/step - loss: 0.3295 - val_loss: 0.3970
Epoch 5/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3269 - val_loss: 0.3998
Epoch 6/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3261 - val_loss: 0.4012
Epoch 7/50
171/171 [==============================] - 2s 12ms/step - loss: 0.3249 - val_loss: 0.4049
Epoch 8/50
171/171 [==============================] - 2s 12ms/step - loss: 0.3245 - val_loss: 0.4016
Epoch 9/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3219 - val_loss: 0.4039
Epoch 10/50
171/171 [==============================] - 2s 13ms/step - loss: 0.3199 - val_lo

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [ ]:
overall_score = 0
for col_index, col in enumerate(target_cols):
        overall_score += spearmanr(preds[:, col_index], val_y[col].values).correlation/len(target_cols)
        print(col, spearmanr(preds[:, col_index], val_y[col].values).correlation)
        fold_scores.append(overall_score)

test_preds += model.predict(test_features)/num_folds
    
print(fold_scores)

question_asker_intent_understanding 0.2628159877204219
question_body_critical 0.5809359669716995
question_conversational 0.30010682539233874
question_expect_short_answer 0.1635210350007904
question_fact_seeking 0.18528088643967305
question_has_commonly_accepted_answer 0.2167923960604781
question_interestingness_others 0.23481103000516035
question_interestingness_self 0.3100467818112976
question_multi_intent 0.35470509047350146
question_not_really_a_question 0.07691308312960277
question_opinion_seeking 0.2875260498973822
question_type_choice 0.46262829309234305
question_type_compare 0.18349088351431675
question_type_consequence 0.1461272430912444
question_type_definition 0.3100052533378793
question_type_entity 0.3824588176796455
question_type_instructions 0.6552175311207464
question_type_procedure 0.2838890090518135
question_type_reason_explanation 0.48577520705972255
question_type_spelling nan
question_well_written 0.4256858559835739
answer_helpful 0.10051895808330759
answer_level_of_i